# YOLOv7 Brain Tumor Segmentation Pipeline
This Colab notebook trains and evaluates YOLOv7 on brain tumor MRI images.

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
import os
import glob

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
base_path = '/content/drive/MyDrive/yolov7_tumor_segmentation'
train_path = f"{base_path}/train"
val_path = f"{base_path}/validation"
test_path = f"{base_path}/test"

# Verify dataset structure
assert os.path.exists(f"{train_path}/images"), "Train images folder not found!"
assert os.path.exists(f"{train_path}/labels"), "Train labels folder not found!"
assert os.path.exists(f"{val_path}/images"), "Validation images folder not found!"
assert os.path.exists(f"{val_path}/labels"), "Validation labels folder not found!"

print("Dataset structure verified!")

In [ ]:
# Step 2: Clone YOLOv7 Repository and Install Dependencies
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt

In [ ]:
# Step 3: Add Empty Labels for Non-Tumor Images
def add_empty_labels(images_path, labels_path):
    # Ensure the labels directory exists
    os.makedirs(labels_path, exist_ok=True)

    # Loop through all images in the images folder
    for image_file in os.listdir(images_path):
        if image_file.endswith(('.jpg', '.png')):
            # Generate the corresponding .txt file path
            label_file = os.path.splitext(image_file)[0] + '.txt'
            label_path = os.path.join(labels_path, label_file)

            # Create an empty .txt file if it doesn't already exist
            if not os.path.exists(label_path):
                open(label_path, 'w').close()  # Create an empty file

# Add empty labels for train, val, and test datasets
add_empty_labels(f"{train_path}/images", f"{train_path}/labels")
add_empty_labels(f"{val_path}/images", f"{val_path}/labels")
add_empty_labels(f"{test_path}/images", f"{test_path}/labels")

print("Empty label files created for non-tumor images!")

In [ ]:
# Step 4: Create Dataset Configuration File
import yaml

data_config = {
    "train": f"{train_path}/images",
    "val": f"{val_path}/images",
    "nc": 1,  # Number of classes (only tumor)
    "names": ["tumor"],  # Class names
}

# Save the dataset configuration file
config_path = f"{base_path}/tumor.yaml"
with open(config_path, "w") as f:
    yaml.dump(data_config, f)

print(f"Dataset configuration file created at: {config_path}")

In [ ]:
# Step 5: Download YOLOv7 Pretrained Weights
weights_path = f"{base_path}/yolov7.pt"
!wget -O {weights_path} https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
# Step 6: Train the Model
print("Starting training...")
!python train.py --weights {weights_path} --cfg cfg/training/yolov7.yaml \
                 --data {config_path} --epochs 100 --batch-size 16 \
                 --img-size 640 --project {base_path} --name yolov7_train

In [ ]:
# Step 7: Locate Trained Weights
trained_weights_path = f"{base_path}/yolov7_train/weights/best.pt"

if os.path.exists(trained_weights_path):
    print(f"Trained weights found at: {trained_weights_path}")
else:
    print("Searching for weights in default directories...")
    best_pt_paths = glob.glob("runs/train/**/weights/best.pt", recursive=True)
    if best_pt_paths:
        trained_weights_path = best_pt_paths[0]
        print(f"Trained weights found at: {trained_weights_path}")
    else:
        raise FileNotFoundError("Trained weights not found! Check training logs.")

In [ ]:
# Step 8: Evaluate the Model
print("Evaluating the model...")
!python test.py --data {config_path} --weights {trained_weights_path} \
                --img-size 640 --project {base_path} --name yolov7_test

In [ ]:
# Step 9: Perform Inference on Test Data
print("Running inference...")
!python detect.py --weights {trained_weights_path} --conf-thres 0.25 \
                  --img-size 640 --source {test_path}/images --project {base_path} \
                  --name yolov7_inference

print(f"Inference results saved to: {base_path}/yolov7_inference")

In [ ]:
# Step 10: Display Output Images in Colab
import matplotlib.pyplot as plt
import cv2

# Display inference results
inference_results_path = f"{base_path}/yolov7_inference"
output_images = glob.glob(f"{inference_results_path}/*.jpg")

print("Displaying Inference Results:")

for img_path in output_images[:5]:  # Display first 5 images
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()